In [8]:
import pandas as pd
import numpy as np

file='ENT_CA_DM_ETOH.txt'

df=pd.read_csv(file,delimiter='|',error_bad_lines=False)
print (df.columns)
cols=['mrn','HX_DRINKS_WK_1','HX_DRINKS_WK_2','HX_DRINKS_WK_3']
df=df[cols]
print ('number of patients:',len(df.mrn.unique()))
print (df)

Index(['mrn', 'HX_DRINK_TYPES_C_1', 'HX_DRINK_TYPES_1', 'HX_DRINKS_WK_1',
       'HX_DRINK_TYPES_C_2', 'HX_DRINK_TYPES_2', 'HX_DRINKS_WK_2',
       'HX_DRINK_TYPES_C_3', 'HX_DRINK_TYPES_3', 'HX_DRINKS_WK_3'],
      dtype='object')
number of patients: 301
         mrn HX_DRINKS_WK_1 HX_DRINKS_WK_2  HX_DRINKS_WK_3
0    1020909              0            NaN             NaN
1    1071436              1            NaN             NaN
2    1156307              5            NaN             NaN
3    1190522              2            NaN             NaN
4    1242125              0            NaN             NaN
..       ...            ...            ...             ...
331   475506              4            NaN             NaN
332   746451              1            NaN             NaN
333   962764             21            NaN             NaN
334   962764             21            NaN             NaN
335   970932              2            NaN             NaN

[336 rows x 4 columns]


In [9]:
def split_the_diff(amount):
    try:
        text=amount.strip()
        num1=int(text.split('-')[0])
        num2=int(text.split('-')[1])
        avg=(num1+num2)/2
        return avg
    except:
        try:
            return int(amount)
        except:
            return np.nan
df['HX_DRINKS_WK_1']=df['HX_DRINKS_WK_1'].apply(split_the_diff)
df['HX_DRINKS_WK_2']=df['HX_DRINKS_WK_2'].apply(split_the_diff)
df['HX_DRINKS_WK_3']=df['HX_DRINKS_WK_3'].apply(split_the_diff)
df=df.fillna(value=0)

df['drinks_per_week']=df[['HX_DRINKS_WK_1','HX_DRINKS_WK_2','HX_DRINKS_WK_3']].sum(axis=1)
print (df.head())

       mrn  HX_DRINKS_WK_1  HX_DRINKS_WK_2  HX_DRINKS_WK_3  drinks_per_week
0  1020909             0.0             0.0             0.0              0.0
1  1071436             1.0             0.0             0.0              1.0
2  1156307             5.0             0.0             0.0              5.0
3  1190522             2.0             0.0             0.0              2.0
4  1242125             0.0             0.0             0.0              0.0


In [10]:
### get rid of duplicates
df=df.sort_values(by=['drinks_per_week'],ascending=False)
print ('size before dropping duplicates:',df.shape[0])
df=df.drop_duplicates(subset='mrn',keep='first')
print ('size after dropping duplicates:',df.shape[0])

### classify high-risk drinking
cut_off=14
df['high_risk']=df['drinks_per_week'].apply(lambda l:l>=cut_off)

print (df)

size before dropping duplicates: 336
size after dropping duplicates: 301
         mrn  HX_DRINKS_WK_1  HX_DRINKS_WK_2  HX_DRINKS_WK_3  drinks_per_week  \
31   2323296            84.0             0.0             0.0             84.0   
132  3192573            70.0             0.0             0.0             70.0   
15   1765632            70.0             0.0             0.0             70.0   
268  3519710            49.0             0.0             0.0             49.0   
290  3554753            14.0            28.0             0.0             42.0   
..       ...             ...             ...             ...              ...   
58   2812904             0.0             0.0             0.0              0.0   
262  3499178             0.0             0.0             0.0              0.0   
50   2738829             0.0             0.0             0.0              0.0   
148  3242778             0.0             0.0             0.0              0.0   
0    1020909             0.0        

In [12]:
print (df['high_risk'].value_counts())
df.to_csv('RM_EtOH.csv',index=False)

False    253
True      48
Name: high_risk, dtype: int64
